# Data Owner - Domain Management

## Setup Variables

Before we start let's some variables that will come in handy later in the notebook.

In [7]:
# Set the Ip Address of your Domain node in this variable.
DOMAIN_HOST_IP="52.157.8.193"

In [24]:
# Set the Ip Address of your Network node in this variable.
# The network node with IP `13.83.1.243` is managed and maintained by OpenMined.
NETWORK_IP="20.121.101.41"

In [10]:
# Set the email and password of your Domain node.
# We will using the default email and password that got created during Domain creation.
# Please update the email and password below incase you update them.

ADMIN_EMAIL="info@openmined.org"
ADMIN_PASSWORD="changethis"

## Install Packages

Let's proceed to install the required packages for accessing the Domain via python.
We will be installing the `syft` library. Syft can be installed using the python `pip` package as follows:

```bash
    pip install 'git+https://github.com/OpenMined/PySyft@dev#egg=syft&subdirectory=packages/syft

```
You may choose to skip this step if you already have the latest version of `syft` package installed in your environment.


**Note:** `syft` is currently supported with python versions **3.8 and above**. Older versions may work, however we have stopped testing and supporting them. If you want to further know on how you can install `syft` in a virtual environment based on your operating system, please refer to the [docs](https://openmined.github.io/PySyft/getting_started/index.html).

Let's check if syft is installed successfully.

In [4]:
import syft as sy
print(f"You're running syft version: {sy.__version__}")

You're running syft version: 0.7.0-beta.14


## Log into the Domain

Now that we have successfully installed `syft`, let's move on to how we can log into a Domain login. There are two ways to log into your own node, as the Data Owner.

1. Using the PySyft library
2. Using the Web Interface

### Using the PySyft library

Let's use the `syft` library to login in to your domain and get an authenticated client to your Domain node.

To login into the your domain you need the following credentials:
- url to the domain: Here your `DOMAIN_HOST_IP` is the url to your domain.
- email address used to signup on the domain: We will use the default email (`ADMIN_EMAIL`) set on domain creation
- password used to signup on the domain: We will use the  default password (`ADMIN_PASSWORD`) set on domain creation
- port number (defaults to 80)

In [ ]:
# Let's log into the domain using the credentials
try:
    domain_client = sy.login(url=DOMAIN_HOST_IP, email=ADMIN_EMAIL, password=ADMIN_PASSWORD)
except Exception as e:
    print("Unable to connect, did you set the `DOMAIN_HOST_IP` variable above?")
    raise e

### Using the Web Interface

We can access the domain node using a Web Interface via the Ip Address defined in the variable `DOMAIN_HOST_IP`.

One can simply access the UI the replacing `DOMAIN_HOST_IP` with your Domain IP Address.

```python
    http://<your_domain_host_ip>/login/
```

To login into the your domain you will need the following credentials:
- email address used to signup on the domain: We will use the default email (`info@openmined.org`) set on domain creation
- password used to signup on the domain: We will use the  default password (`changethis`) set on domain creation


On accessing the url defined above you should be able to see the login page as shown in the image below.

![Domain Login Page](img/pygrid_ui_login.png)

On a sucessful login you will be redirected the users page, where you can manage all the that are users signed up to your domain.

## Connect to a Network

In [55]:
network_client = sy.login(url=NETWORK_IP)

Connecting to 13.83.1.243... done! 	 Logging into admiring_bostrom... as GUEST...done!

**Warning**: The syft version on your system and the node are different.
Version on your system: 0.7.0-beta.9
Version on the node: 0.7.0-beta.8



In [56]:
# see what domains are on a network

In [57]:
network_client.domains

""


## Log into your Domain

In [37]:
try:
    domain_client = sy.login(url=HOST_IP, email=ADMIN_EMAIL, password=ADMIN_PASSWORD)
except Exception:
    print("Unable to connect, did you set the HOST_IP variable above?")


Anyone can login as an admin to your node right now because your password is still the default PySyft username and password!!!

Connecting to 52.157.8.193... done! 	 Logging into modest_hutter... done!

**Warning**: The syft version on your system and the node are different.
Version on your system: 0.7.0-beta.9
Version on the node: 0.7.0-beta.8



In [7]:
# what can a domain client do?

In [58]:
domain_client.store

,ID,Tags,Description,object_type
0,<UID: d39e8808c77644dc8d58a31b6e97d20b>,[#tensor_1],,<class 'syft.core.tensor.tensor.Tensor'>


## Join the Network

In [60]:
domain_client.apply_to_network(network_client)

🔌 <DomainClient - modest_hutter: <UID: 451739f8fb054fab9c83f6422156481e>> successfully connected to the VPN: http://13.83.1.243:80/api/v1


In [67]:
network_client.domains

""


In [63]:
domain_client.vpn_status()

{'status': 'ok',
 'connected': True,
 'host': {'ip': '100.64.0.2',
  'hostname': 'modest_hutter',
  'network': 'omnet',
  'os': 'linux',
  'connection_info': '-',
  'connection_status': 'n/a',
  'connection_type': 'n/a'},
 'peers': [{'ip': '100.64.0.1',
   'hostname': 'admiring_bostrom',
   'network': 'omnet',
   'os': 'linux',
   'connection_info': 'active; direct 13.83.1.243:41641; offline, tx 1388 rx 1384',
   'connection_status': 'active',
   'connection_type': 'direct'}]}

## Create a Dataset

In [66]:
example_data = [1, 2, 3]
example_tensor = sy.Tensor(example_data)

In [10]:
(example_tensor, example_tensor.public_shape)

(Tensor(child=[1 2 3]), (3,))

In [11]:
# make your tensor private with min, max and data subjects

In [12]:
private_tensor = example_tensor.private(
    min_val=0, max_val=3, entities=[sy.core.adp.entity.Entity("0")]
).tag("example_data")

In [13]:
# create a dataset

In [14]:
domain_client.load_dataset(
    assets={"tensor_1": private_tensor},
    name="My Private Data",
    description=("A collection of medical images"),
    skip_checks=True,
)

Loading dataset... uploading... SUCCESS!                                                                                                                                      

Run <your client variable>.datasets to see your new dataset loaded into your machine!


In [15]:
domain_client.datasets

Idx,Name,Description,Assets,Id
[0],My Private Data,A collection of medical images,"[""tensor_1""] -> int32",82168158-eec9-4f61-b3da-647372b62a3c


## Create a Data Scientist Account

In [17]:
data_scientist_details = {
    "name": "Samantha Carter",
    "email": "sam@sg1.net",
    "password": "stargate",
    "budget": 9999,
}

In [18]:
domain_client.users.create(**data_scientist_details)

In [19]:
print("Please give these details to the data scientist:")
login_details = {}
login_details["url"] = HOST_IP
login_details["name"] = data_scientist_details["name"]
login_details["email"] = data_scientist_details["email"]
login_details["password"] = data_scientist_details["password"]
print(login_details)

Please give these details to the data scientist:
{'url': '52.157.8.193', 'name': 'Samantha Carter', 'email': 'sam@sg1.net', 'password': 'stargate'}
